# Define the network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0784,  0.0446,  0.0360,  0.0864, -0.0798,  0.0229,  0.0531,  0.0290,
          0.0556, -0.0693]], grad_fn=<AddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

# Loss Function

In [14]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
print(target)
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
target

tensor([-1.5730,  0.1748, -0.9840,  1.7351, -0.1026, -0.1544, -0.6448, -0.6458,
        -0.3423,  0.4047])


tensor([[-1.5730,  0.1748, -0.9840,  1.7351, -0.1026, -0.1544, -0.6448, -0.6458,
         -0.3423,  0.4047]])

In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# Backprop

In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 3.4424e-03,  1.0096e-02, -6.9098e-03, -4.9714e-03,  9.2072e-03,
         3.6719e-05])


# Update the weights

In [9]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
    print(f)

Parameter containing:
tensor([[[[-2.5497e-01,  2.4802e-01, -2.7424e-01],
          [-8.5760e-02,  1.0917e-01,  2.3849e-01],
          [ 3.2183e-01,  3.1253e-01, -1.5622e-01]]],


        [[[ 2.6198e-01, -4.1015e-03, -2.4888e-01],
          [ 1.4990e-02, -2.3546e-01,  2.7228e-01],
          [ 2.8935e-01, -2.3341e-01, -1.4190e-01]]],


        [[[ 1.7328e-01, -1.9965e-01, -8.7381e-02],
          [ 3.5723e-02,  4.2177e-02,  1.2135e-01],
          [ 1.2387e-01, -3.1349e-01,  2.5249e-01]]],


        [[[ 3.2586e-01, -3.9909e-02, -7.6683e-02],
          [-3.0601e-01, -8.4452e-02, -1.3546e-01],
          [ 2.0413e-01, -2.0206e-01, -2.0970e-01]]],


        [[[-2.8684e-01, -2.1092e-01, -1.9876e-01],
          [ 1.0896e-01, -1.9501e-03, -1.7771e-02],
          [ 7.2227e-02,  2.3185e-01,  2.1584e-01]]],


        [[[ 1.2789e-01, -6.0161e-02,  2.8277e-04],
          [-1.8499e-01,  1.3484e-01,  2.2714e-01],
          [ 1.5114e-01,  1.3655e-01,  1.4067e-01]]]], requires_grad=True)
Parameter contain

In [9]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update